<a href="https://colab.research.google.com/github/lailatulqadri/MusicGenreIdentifier/blob/main/TA2023_Text_Analytic_Applications_Document_Clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

We will use nltk toolkit to assist text cleaning.

In [49]:
#To import and download nltk
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

We use sklearn library for machine learning based analysis. Sklearn support many machine learning approaches such as SVM, nearest neighbour, random forest, k-means and many more. You can find more information about sklearn in this webpage: https://scikit-learn.org/stable/

In [50]:
# Importing libraries
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cluster import KMeans
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import string
import re
import numpy as np
from collections import Counter

stop = set(stopwords.words('english'))
exclude = set(string.punctuation)
lemma = WordNetLemmatizer()



Yup, when you work with text, text cleaning is a crucial task! This simple function allows you to clean doc. You can modify this function to fit your text cleaning processes.

In [51]:
# Cleaning the text sentences so that punctuation marks, stop words & digits are removed
def clean(doc):
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
    processed = re.sub(r"\d+","",normalized)
    y = processed.split()
    return y

Document Representations used in this example is Tf-Idf. You already learn this method last 2 weeks.

---
Term frequency and inverse document frequency (tf-idf) is a word weighing scheme. The tf-idf value increases as the number of times a word appears in a document. So, it helps to reflect the importance of a word in a corpus.


In [52]:
import pandas as pd

Describing the information/categories in the dataset. The dataset is not labeled.

In [53]:
print ("There are 10 sentences of following three classes on which K-NN classification and K-means clustering"\
         " is performed : \n1. Cricket \n2. Artificial Intelligence \n3. Chemistry")


There are 10 sentences of following three classes on which K-NN classification and K-means clustering is performed : 
1. Cricket 
2. Artificial Intelligence 
3. Chemistry


Let's read our data!

In [54]:
# read traning dataset
dataset = pd.read_csv('Clustering dataset.csv')
print("Data\n",dataset)

train_clean_sentences = []


Data
                                                  Text
0   Cricket is a bat and ball game played between ...
1   Each phase of play is called an innings during...
2   The teams have one or two innings apiece and, ...
3   Before a match begins, the two team captains m...
4   Two batsmen and eleven fielders then enter the...
5    The most common dismissal in cricket match ar...
6   Runs are scored by two main methods: either by...
7   The main objective of each team is to score mo...
8   If the team batting last is all out having sco...
9   The role of striker batsman is to prevent the ...
10  Artificial intelligence is intelligence exhibi...
11  The field of AI research defines itself as the...
12  The overall research goal of artificial intell...
13  Natural language processing[77] gives machines...
14  AI researchers developed sophisticated mathema...
15  An intelligent agent is a system that perceive...
16  AI techniques have become an essential part of...
17  Recent advancement

Text Preprocessing - for each document in our dataset, we will give it to clean function above. The document can be found in dataset['Text'] <- inside column 'Text'

In [55]:
for line in dataset["Text"]:
    line = line.strip()
    cleaned = clean(line)
    cleaned = ' '.join(cleaned)
    train_clean_sentences.append(cleaned)

Output after text preprocessing

In [56]:
print(train_clean_sentences)

['cricket bat ball game played two team eleven player cricket field', 'phase play called inning one team bat attempting score many run possible', 'team one two inning apiece and first inning end team swap role next inning', 'match begin two team captain meet pitch toss coin determine team bat first', 'two batsman eleven fielder enter field play begin member fielding team known bowler delivers ball', 'common dismissal cricket match bowled bowler hit stump directly ball dislodges bail batsman get out', 'run scored two main method either hitting ball hard enough cross boundary two batsman swapping end', 'main objective team score run opponent', 'team batting last scored fewer run opponent said lost n run', 'role striker batsman prevent ball hitting stump using bat and simultaneously strike well enough score run', 'artificial intelligence intelligence exhibited machine rather human animal', 'field ai research defines study intelligent agent device perceives environment take action maximize

Text representation by using TFIDF

In [57]:
vectorizer = TfidfVectorizer(stop_words='english')
X = vectorizer.fit_transform(train_clean_sentences)


Let's split the data into training our model. For easy understanding, we will label our dataset to 0, 1 & 2.

In [58]:
# Creating true labels for 30 training sentences
y_train = np.zeros(30)  # label 0
y_train[10:20] = 1      # label 1
y_train[20:30] = 2      # label 2

# 3. Training the Classification (K-NN) & Clustering (K-Means) models.

---
Output for k-NN classification is category membership. Texts are assigned to class that is most common among their k nearest neighbours.

Clustering determines the grouping of a set of data.In K-Means clustering, the 'K' cluster centers are the centroid of data points of a cluster.


In [59]:
# Classification the document with KNN classifier
modelknn = KNeighborsClassifier(n_neighbors=5)
modelknn.fit(X,y_train)



KNeighborsClassifier()

In [60]:
# Clustering the training 30 sentences with K-means technique
modelkmeans = KMeans(n_clusters=3, init='k-means++', max_iter=200, n_init=100)
modelkmeans.fit(X)

KMeans(max_iter=200, n_clusters=3, n_init=100)

Testing on Unseen Texts.

---
This is to test the trained model. It is tested using 3 text sentences.


In [61]:
# Predicting it on test data : Testing Phase
test_sentences = ["Chemical compunds are used for preparing bombs based on some reactions",\
"Cricket is a boring game where the batsman only enjoys the game",\
"Machine learning is a area of Artificial intelligence"]

test_clean_sentence = []
for test in test_sentences:
  cleaned_test = clean(test)
  cleaned = ' '.join(cleaned_test)
  cleaned = re.sub(r"\d+","",cleaned)
  test_clean_sentence.append(cleaned)

Test = vectorizer.transform(test_clean_sentence)

true_test_labels = ['Cricket','AI','Chemistry']
#classification
predicted_labels_knn = modelknn.predict(Test)
#clustering
predicted_labels_kmeans = modelkmeans.predict(Test)

print ("\nBelow 3 sentences will be predicted against the learned nieghbourhood and learned clusters:\n1. ",\
test_sentences[0],"\n2. ",test_sentences[1],"\n3. ",test_sentences[2])
print ("\n-------------------------------PREDICTIONS BY KNN------------------------------------------")
print ("\n",test_sentences[0],":",true_test_labels[np.int(predicted_labels_knn[0])],\
"\n",test_sentences[1],":",true_test_labels[np.int(predicted_labels_knn[1])],\
"\n",test_sentences[2],":",true_test_labels[np.int(predicted_labels_knn[2])])

print ("\n-------------------------------PREDICTIONS BY K-Means--------------------------------------")
print ("\nIndex of Cricket cluster : ",Counter(modelkmeans.labels_[0:10]).most_common(1)[0][0])
print ("Index of Artificial Intelligence cluster : ",Counter(modelkmeans.labels_[10:20]).most_common(1)[0][0])
print ("Index of Chemistry cluster : ",Counter(modelkmeans.labels_[20:30]).most_common(1)[0][0])

print ("\n",test_sentences[0],":",predicted_labels_kmeans[0],\
"\n",test_sentences[1],":",predicted_labels_kmeans[1],\
"\n",test_sentences[2],":",predicted_labels_kmeans[2])


Below 3 sentences will be predicted against the learned nieghbourhood and learned clusters:
1.  Chemical compunds are used for preparing bombs based on some reactions 
2.  Cricket is a boring game where the batsman only enjoys the game 
3.  Machine learning is a area of Artificial intelligence

-------------------------------PREDICTIONS BY KNN------------------------------------------

 Chemical compunds are used for preparing bombs based on some reactions : Chemistry 
 Cricket is a boring game where the batsman only enjoys the game : Cricket 
 Machine learning is a area of Artificial intelligence : AI

-------------------------------PREDICTIONS BY K-Means--------------------------------------

Index of Cricket cluster :  2
Index of Artificial Intelligence cluster :  0
Index of Chemistry cluster :  1

 Chemical compunds are used for preparing bombs based on some reactions : 1 
 Cricket is a boring game where the batsman only enjoys the game : 2 
 Machine learning is a area of Artifici

<ipython-input-61-64aad570e0a7>:24: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  print ("\n",test_sentences[0],":",true_test_labels[np.int(predicted_labels_knn[0])],\
<ipython-input-61-64aad570e0a7>:25: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for ad

From the output above, we can see that all of the 3 sentences tested are correctly clustered and classified into categories.